# MNIST 실습

- training set: 60,000장
- test set: 10,0000장
- image resolution: 28*28, grayscale(=1 channel) 
- value range: 0-255
- label: from 0 to 9, total 10 label for each images

In [1]:
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist  # 이 ipynb 파일의 폴더 아래에 data폴더속 mnist.py 파일안에 load_mnist 함수가 정의되어 있다

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

# 각 데이터 모양 출력
print(x_train.shape)  # (60000, 784), 784는 28*28
print(t_train.shape)  # (60000, )
print(x_test.shape)  # (10000, 784)
print(t_test.shape)  # (10000, )

Done
Done
Done
Done
Converting train-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting train-labels-idx1-ubyte.gz to NumPy Array ...
Done
Converting t10k-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting t10k-labels-idx1-ubyte.gz to NumPy Array ...
Done
Creating pickle file ...
Done!
(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [2]:
# mnist image를 화면에 뿌려보자

import numpy as np
from PIL import Image  # Python Image Library

def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()
    
img = x_train[0]
label = t_train[0]
print(label)

print(img.shape)  # flattened from 1*28*28 = (784, )
img = img.reshape(28,28)
print(img.shape)

img_show(img)

5
(784,)
(28, 28)


In [15]:
# 실제로 neural networks를 구현해보자
import pickle
import numpy as np
#from common.functions import sigmoid, softmax  # 이건 어짜피 우리가 만들어둔 함수들임
#from sandbox import sigmoid, softmax  # 이건 어짜피 우리가 만들어둔 함수들임
def sigmoid(x):
    return 1/(1 + exp(-x))

def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a/sum_exp_a
    #print(y)
    return y

def softmax2(a):
    #print(np.exp(a)/np.sum(np.exp(a)))
    return np.exp(a)/np.sum(np.exp(a))

def get_data():
    """
    normalize (= 0-1안의 값으로 바꾸기) 하고 
    flattern (=vector로 바꾸기)한 값으로 읽어와서 test set만 리턴
    """
    (x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=True, one_hot_label=False)
    return x_test, t_test

def init_network():
    """
    W 값들의 초기화?
    """
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
        
    return network

def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)
    
    return y

In [16]:
x, t = get_data()
network = init_network()  # 이 값은 sample_weight.pkl 파일의 W와 b 값인데 결국 이녀석은 미리 train 되어 있는 값인 것이다.

accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y)
    if p == t[i]:
        accuracy_cnt +=1
        
print("Accuracy: " + str(float(accuracy_cnt) / len(x)))

Accuracy: 0.9352


In [19]:
# 위 셀과 같은 기능을 배치 처리로 해보자. 
# 이미지 한장씩이 아니라 한방에 다 올리자는 것
x, t = get_data()
network = init_network()

batch_size = 100
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])
    
print("Accuracy: " + str(float(accuracy_cnt) / len(x)))

Accuracy: 0.9352


In [21]:
# axis의 개념을 이해해보자
# axis는 0, 1로 커진다

x = np.array([[1,2,3], [4,5,6], [7,8,9], [10,11,12]])

# numpy.argmax() 는 가장 큰 녀석의 index 값을 리턴해준다
y = np.argmax(x, axis = 1)
z = np.argmax(x, axis = 0)
print(x)
print(y)
print(z)

[[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]]
[2 2 2 2]
[3 3 3]
